John Lubisich

15 October 2024

# NLP Hobby Recommender

## Introduction

For any individual, a hobby that is good fit to their lifestyles or preferences can benefit their mental health, physical health, and/or be a great way to relax. From this, hobbies can be seen as very important to those who struggle with any or all of these things. However, hobbies include many other characteristics such as cost, seasonality, time commitment, etc. Therefore, it can be difficult to find a hobby that is a perfect fit for a particular person's lifestyle.

## Purpose

Considering the importance of hobbies and the difficult of finding a perfect fit, we will create a content based recommendation system that takes a hobby that a person knows they enjoy and pair them to similar but different hobbies, in hopes that one can better fit in their life. This is especially important for those who have recently undergone change in their life and need to find another hobby. For example, if someone has undergone an injury and cannot perform their favorite hobby anymore, using this tool they will be able to enter their favorite hobby and be given options of similar hobbies with descriptions of how the hobbies differ. The user would then be able to use their own discretion and pick a recommended hobby that is well suited to themselves.

# The Code

## Standard Libraries

We will now import the necessary standard libraries.

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.metrics.pairwise import cosine_similarity

# Inserting our Data

We will now use our hobbies data with information we learned from reseaching each hobbies and the components that go into performing each hobby.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
hobbies = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hobbies.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
hobbies=hobbies.drop("Category", axis=1)

In [ ]:
list(hobbies.columns.values)

['Hobby',
 'Description',
 'Time commitment',
 'Material Requirement',
 'Achievement',
 'Mental Stimulation',
 'Discovery',
 'Health benefits',
 'Physical intensity',
 'Cost',
 'Seasonal',
 'Indoor/Outdoor']

Above, we can see the components that we consider the most important when attempting to find a perfect fit hobby. We have the hobby name and a short description of the hobby. Also, on a scale from 1 to 5 (5 being the greatest), we have given each hobby what we believe to be fit in its required time commitment, material requirement, achievement posibility, mental stimulation, discovery possibility, health benefits, physical intensity, and cost. We have also provided whether the hobby is seasonal or not as well as indoor or outdoor, so the user of this program can consider finding a different hobby during the post-seasonal time of this hobby or on a rainy day.

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

hobbies['Processed_Description'] = hobbies['Description'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(hobbies['Processed_Description'])


In [ ]:
from scipy.sparse import hstack

hobbies['Seasonal'] = hobbies['Seasonal'].map({'yes': 1, 'no': 0})
hobbies['Indoor/Outdoor'] = hobbies['Indoor/Outdoor'].map({'indoor': 0, 'outdoor': 1, 'either': 2})

numerical_features = hobbies[['Material Requirement', 'Achievement', 'Mental Stimulation', 'Discovery', 'Time commitment', 'Health benefits', 'Physical intensity', 'Cost', 'Seasonal', 'Indoor/Outdoor']].values
combined_features = hstack([tfidf_matrix, numerical_features])

combined_features.shape


(59, 1028)

In [ ]:
similarity_matrix = cosine_similarity(combined_features)

In [ ]:
def recommend_hobbies(hobby_name):
    top_n=5
    if hobby_name not in hobbies['Hobby'].values:
        return "Hobby not found in the dataset."

    idx = hobbies[hobbies['Hobby'] == hobby_name].index[0]

    similarity_scores = similarity_matrix[idx]


    similar_hobbies_idx = similarity_scores.argsort()[::-1]

    top_hobbies = []

    for i in similar_hobbies_idx:
        if i != idx:
            top_hobbies.append(hobbies['Hobby'].iloc[i])
        if len(top_hobbies) == top_n:
            break

    return top_hobbies

## Recommendations

Using our system, we will give some example recommendations for hobbies.

In [ ]:
recommend_hobbies('Woodworking')

['Metalworking', 'Sewing', 'Embroidery', 'Knitting', 'Beading']

In [ ]:
recommend_hobbies('Dancing')

['Tai Chi', 'Martial Arts', 'Running', 'Acting', 'Singing']

In [ ]:
recommend_hobbies('Yoga')

['Meditation', 'Journalism', 'Writing', 'Puzzling', 'Acting']

After using our tool, we have new recommendations for Hobbies, similar to our examples used. With this, we can now branch out and try hobbies that we are likely to enjoy as well as fit nicely in our characteristics.

## Recomendation Summaries

To better give our users an idea of their recommended hobbies, we will provide them with a description and the components of the hobby. This is important to assist users in finding the hobby fit for them after considering the components of each hobby. For the sake of space, we will be using the recommended hobbies for Blacksmith. Note: For Seasonal, 1 = yes and 0 = no and for Indoor/Outdoor,
0 = indoor, 1 = outdoor, and 2 = either.

In [ ]:
hobbies[hobbies['Hobby']=='Metalworking']['Description'].item()

'Blacksmithing is an ancient craft that involves shaping and forging metal using heat, hammers, and anvils to create tools, weapons, decorative items, or even sculptures. As a hobby, blacksmithing combines artistry with physical skill, allowing you to create custom, handmade pieces from metal. It requires learning how to control heat, manipulate materials, and use various tools to achieve the desired shape and design. Blacksmithing is both a meditative and physically demanding craft, and it offers the satisfaction of creating functional art that can last for generations.'

In [ ]:
hobbies[hobbies['Hobby']=='Metalworking'].iloc[0]

,50
Hobby,Metalworking
Description,Blacksmithing is an ancient craft that involve...
Time commitment,4
Material Requirement,5
Achievement,4
Mental Stimulation,3
Discovery,1
Health benefits,2
Physical intensity,5
Cost,5


In [ ]:
hobbies[hobbies['Hobby']=='Sewing']['Description'].item()

"Sewing is a versatile and practical hobby that involves stitching fabric together to create clothing, accessories, home decor, or even quilts. Whether you're hand-sewing or using a sewing machine, it allows you to express your creativity through design, pattern-making, and fabric selection. Sewing can be both relaxing and productive, offering the satisfaction of creating something useful or beautiful from scratch. As a hobby, it also provides the opportunity to learn new skills, like tailoring, embroidery, or upcycling old garments into new creations."

In [ ]:
hobbies[hobbies['Hobby']=='Sewing'].iloc[0]

,52
Hobby,Sewing
Description,Sewing is a versatile and practical hobby that...
Time commitment,3
Material Requirement,4
Achievement,3
Mental Stimulation,2
Discovery,1
Health benefits,1
Physical intensity,4
Cost,4


In [ ]:
hobbies[hobbies['Hobby']=='Embroidery']['Description'].item()

"Embroidery is a creative hobby that involves decorating fabric using thread or yarn to create intricate patterns, designs, or even images. It can be done by hand or with a sewing machine and can range from simple stitches to highly detailed artwork. Whether you're embellishing clothes, creating wall art, or making custom gifts, embroidery allows for endless creativity and personalization. It's a relaxing and rewarding craft that can be both meditative and expressive, helping to improve fine motor skills and attention to detail."

In [ ]:
hobbies[hobbies['Hobby']=='Embroidery'].iloc[0]

,51
Hobby,Embroidery
Description,Embroidery is a creative hobby that involves d...
Time commitment,2
Material Requirement,3
Achievement,3
Mental Stimulation,2
Discovery,1
Health benefits,1
Physical intensity,3
Cost,3


In [ ]:
hobbies[hobbies['Hobby']=='Knitting']['Description'].item()

'Knitting as a hobby is a relaxing and creative way to craft unique, handmade items like scarves, sweaters, or blankets. It combines focus and patience, allowing you to unwind while creating something beautiful and functional. The rhythmic motion of knitting can be meditative, and the sense of accomplishment when finishing a project makes it a satisfying and rewarding activity.'

In [ ]:
hobbies[hobbies['Hobby']=='Knitting'].iloc[0]

,11
Hobby,Knitting
Description,Knitting as a hobby is a relaxing and creative...
Time commitment,2
Material Requirement,4
Achievement,2
Mental Stimulation,2
Discovery,1
Health benefits,1
Physical intensity,2
Cost,3


In [ ]:
hobbies[hobbies['Hobby']=='Beading']['Description'].item()

'Beading is a creative hobby that involves stringing beads together to create jewelry, accessories, or decorative items. It allows you to experiment with various colors, shapes, and materials—such as glass, metal, or wood—to craft unique pieces. Beading can range from simple designs like bracelets and earrings to intricate patterns in necklaces or even bead-embellished clothing. It’s a relaxing and meditative activity that also enhances your fine motor skills, attention to detail, and creativity, making it a rewarding and personalized craft.'

In [ ]:
hobbies[hobbies['Hobby']=='Beading'].iloc[0]

,53
Hobby,Beading
Description,Beading is a creative hobby that involves stri...
Time commitment,2
Material Requirement,3
Achievement,3
Mental Stimulation,2
Discovery,1
Health benefits,1
Physical intensity,1
Cost,3


In this example, if the user had a woodworking hobby in the past the system would recommend similar hobbies to Woodworking. Now, the user can find a fit using the information provided through their recommended hobbies.

# Analysis

All of these recommendations make sense due to several shared charactersitics across these hobbies like, craftmanship, creativity, tools and time commitments. This is important for someone trying to find a new hobby based on one they already enjoy because of the amount of transferable skills each of these recommendations include. Because of the shared similarities, trying one of these new hobbies will be more likely to have a smoother transition and make for a more enjoyable experience.

# Conclusion

This tool is important for those who want to find a new hobby. With this tool, they can now put in a hobby they know that they like and find different but similar hobbies. From this, there may be a hobby that fits better in their lifestyle characteristics, such as their current alloted time, cost, or physical intensity for the hobby. This could be very helpful for college students in particular since each week can look very different from the next. However, it can also work well for those who have recently experienced changes in their lives, such as less time they can commit to a hobby or an injury that prevents them from endulging in their past hobbies.